# Laboratorio 5

## Datos: _European Union lesbian, gay, bisexual and transgender survey (2012)_

Link a los datos [aquí](https://www.kaggle.com/ruslankl/european-union-lgbt-survey-2012).

### Contexto

La FRA (Agencia de Derechos Fundamentales) realizó una encuesta en línea para identificar cómo las personas lesbianas, gays, bisexuales y transgénero (LGBT) que viven en la Unión Europea y Croacia experimentan el cumplimiento de sus derechos fundamentales. La evidencia producida por la encuesta apoyará el desarrollo de leyes y políticas más efectivas para combatir la discriminación, la violencia y el acoso, mejorando la igualdad de trato en toda la sociedad. La necesidad de una encuesta de este tipo en toda la UE se hizo evidente después de la publicación en 2009 del primer informe de la FRA sobre la homofobia y la discriminación por motivos de orientación sexual o identidad de género, que destacó la ausencia de datos comparables. La Comisión Europea solicitó a FRA que recopilara datos comparables en toda la UE sobre este tema. FRA organizó la recopilación de datos en forma de una encuesta en línea que abarca todos los Estados miembros de la UE y Croacia. Los encuestados eran personas mayores de 18 años, que se identifican como lesbianas, homosexuales, bisexuales o transgénero, de forma anónima. La encuesta se hizo disponible en línea, de abril a julio de 2012, en los 23 idiomas oficiales de la UE (excepto irlandés) más catalán, croata, luxemburgués, ruso y turco. En total, 93,079 personas LGBT completaron la encuesta. Los expertos internos de FRA diseñaron la encuesta que fue implementada por Gallup, uno de los líderes del mercado en encuestas a gran escala. Además, organizaciones de la sociedad civil como ILGA-Europa (Región Europea de la Asociación Internacional de Lesbianas, Gays, Bisexuales, Trans e Intersexuales) y Transgender Europe (TGEU) brindaron asesoramiento sobre cómo acercarse mejor a las personas LGBT.

Puede encontrar más información sobre la metodología de la encuesta en el [__Informe técnico de la encuesta LGBT de la UE. Metodología, encuesta en línea, cuestionario y muestra__](https://fra.europa.eu/sites/default/files/eu-lgbt-survey-technical-report_en.pdf).

### Contenido

El conjunto de datos consta de 5 archivos .csv que representan 5 bloques de preguntas: vida cotidiana, discriminación, violencia y acoso, conciencia de los derechos, preguntas específicas de personas transgénero.

El esquema de todas las tablas es idéntico:

* `CountryCode` - name of the country
* `subset` - Lesbian, Gay, Bisexual women, Bisexual men or Transgender (for Transgender Specific Questions table the value is only Transgender)
* `question_code` - unique code ID for the question
* `question_label` - full question text
* `answer` - answer given
* `percentage`
* `notes` - [0]: small sample size; [1]: NA due to small sample size; [2]: missing value

En el laboratorio de hoy solo utilizaremos los relacionados a la vida cotidiana, disponibles en el archivo `LGBT_Survey_DailyLife.csv` dentro de la carpeta `data`.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
daily_life_raw = pd.read_csv(os.path.join("..", "data", "LGBT_Survey_DailyLife.csv"))
daily_life_raw.head()

In [ ]:
daily_life_raw.info()

In [ ]:
daily_life_raw.describe(include="all").T

In [ ]:
questions = (
    daily_life_raw.loc[: , ["question_code", "question_label"]]
    .drop_duplicates()
    .set_index("question_code")
    .squeeze()
)
for idx, value in questions.items():
    print(f"Question code {idx}:\n\n{value}\n\n")

### Preprocesamiento de datos

¿Te fijaste que la columna `percentage` no es numérica? Eso es por los registros con notes `[1]`, por lo que los eliminaremos.

In [ ]:
daily_life_raw.notes.unique()

In [ ]:
daily_life = (
    daily_life_raw.query("notes != ' [1] '")
    .astype({"percentage": "int"})
    .drop(columns=["question_label", "notes"])
    .rename(columns={"CountryCode": "country"})
)
daily_life.head()

## Ejercicio 1

(1 pto)

¿A qué tipo de dato (nominal, ordinal, discreto, continuo) corresponde cada columna del DataFrame `daily_life`?

Recomendación, mira los valores únicos de cada columna.

In [ ]:
daily_life.dtypes

In [ ]:
for c in daily_life.columns:
    print(f"para {c} los datos únicos son: \n\n{daily_life[c].unique()}\n" )

__Respuesta:__

* `country`: Nominal, a priori no hay un orden de países.
* `subset`: Nominal
* `question_code`: Nominal, no es posible señalar si `open_at_school` va antes o después de `g1_a`.
* `answer`: Nominal, no es posible señalar si  `I read about it in a newspaper (online or printed)` va antes o después de  `I received an email from an organisation or online network`, si se consideran las respuestas para cada pregunta en especifico y no la columna entera, la situación puede ser distinta.
* `percentage`: continuo, teniendo en cuenta la naturaleza de la variable(porcentaje).

## Ejercicio 2 

(1 pto)

Crea un nuevo dataframe `df1` tal que solo posea registros de Bélgica, la pregunta con código `b1_b` y que hayan respondido _Very widespread_.

Ahora, crea un gráfico de barras vertical con la función `bar` de `matplotlib`  para mostrar el porcentaje de respuestas por cada grupo. La figura debe ser de tamaño 10 x 6 y el color de las barras verde.

In [ ]:
print(f"Question b1_b:\n\n{questions['b1_b']}")

In [ ]:
df1 = daily_life.loc[lambda df: (df["country"] == "Belgium") & (df["question_code"]=="b1_b")  & (df["answer"]=='Very widespread') ]
df1 #usar query?

In [ ]:
x = df1["subset"]
y = df1["percentage"]

fig = plt.figure(figsize=(10,6))

plt.bar(x,y,color="g")
plt.show()

## Ejercicio 3

(1 pto)

Respecto a la pregunta con código `g5`, ¿Cuál es el porcentaje promedio por cada valor de la respuesta (notar que la respuestas a las preguntas son numéricas)?

In [ ]:
print(f"Question g5:\n\n{questions['g5']}")

Crea un DataFrame llamado `df2` tal que:

1. Solo sean registros con la pregunta con código `g5`
2. Cambia el tipo de la columna `answer` a `int`.
3. Agrupa por país y respuesta y calcula el promedio a la columna porcentaje (usa `agg`).
4. Resetea los índices.

In [ ]:
df2 = (
    daily_life
    .loc[lambda df: df["question_code"]=="g5"]
    .astype({"answer":"int"})
    .groupby(["country","answer"])
    .agg(porcentaje_promedio=("percentage","mean") )
    .reset_index()
)
df2

Crea un DataFrame llamado `df2_mean` tal que:

1. Agrupa `df2` por respuesta y calcula el promedio del porcentaje.
2. Resetea los índices.

In [ ]:
df2_mean = df2.groupby("answer").agg(porcentaje_promedio_respuesta=("porcentaje_promedio","mean")).reset_index()
df2_mean

Ahora, grafica lo siguiente:

1. Una figura con dos columnas, tamaño de figura 15 x 12 y que compartan eje x y eje y. Usar `plt.subplots`.
2. Para el primer _Axe_ (`ax1`), haz un _scatter plot_ tal que el eje x sea los valores de respuestas de `df2`, y el eye y corresponda a los porcentajes de `df2`. Recuerda que en este caso corresponde a promedios por país, por lo que habrán más de 10 puntos en el gráfico..
3. Para el segundo _Axe_ (`ax2`), haz un gráfico de barras horizontal tal que el eje x sea los valores de respuestas de `df2_mean`, y el eye y corresponda a los porcentajes de `df2_mean`. 

In [ ]:
x = df2["answer"]
y = df2["porcentaje_promedio"]

x_mean = df2_mean["answer"]
y_mean = df2_mean["porcentaje_promedio_respuesta"]


fig, (ax1, ax2) = plt.subplots(ncols=2,figsize=(15,12),sharex=True, sharey=True)

ax1.scatter(x,y)
ax1.grid(alpha=0.3)

ax2.barh(y_mean,x_mean)
#ax2.bar(x_mean,y_mean) extra 
ax2.grid(alpha=0.3)

fig.show()

## Ejercicio 4

(1 pto)

Respecto a la misma pregunta `g5`, cómo se distribuyen los porcentajes en promedio para cada país - grupo?

Utilizaremos el mapa de calor presentado en la clase, para ello es necesario procesar un poco los datos para conformar los elementos que se necesitan.

Crea un DataFrame llamado `df3` tal que:

1. Solo sean registros con la pregunta con código `g5`
2. Cambia el tipo de la columna `answer` a `int`.
3. Agrupa por país y subset, luego calcula el promedio a la columna porcentaje (usa `agg`).
4. Resetea los índices.
5. Pivotea tal que los índices sean los países, las columnas los grupos y los valores el promedio de porcentajes.
6. Llena los valores nulos con cero. Usa `fillna`.

In [ ]:
## Code from:
# https://matplotlib.org/3.1.1/gallery/images_contours_and_fields/image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
df3 = (
    daily_life
    .loc[lambda df: df["question_code"]=="g5"]
    .astype({"answer":"int"})
    .groupby(["country","subset"])
    .agg(porcentaje_promedio=("percentage","mean"))
    .reset_index()
    .pivot(index="country",columns="subset",values="porcentaje_promedio")
    .fillna(0)
)
df3.head()

Finalmente, los ingredientes para el heat map son:

In [ ]:
countries = df3.index.tolist()
subsets = df3.columns.tolist()
answers = df3.values

El mapa de calor debe ser de la siguiente manera:

* Tamaño figura: 15 x 20
* cmap = "YlGn"
* cbarlabel = "Porcentaje promedio (%)"
* Precisión en las anotaciones: Flotante con dos decimales.

In [ ]:
fig, ax = plt.subplots(figsize=(15,20))

im, cbar = heatmap(answers,countries,subsets,ax=ax,cmap="YlGn",cbarlabel = "Porcentaje promedio (%)")
texts = annotate_heatmap(im,valfmt="{x:.2f}%")

fig.tight_layout()
plt.show()